# Reserve a Site
This Jupyter notebook provides a step-by-step guide to reserving an entire site for a specified period in the near future. This capability is particularly useful for scheduling maintenance activities, ensuring that resources at the selected site are fully reserved and unavailable for other tasks during the designated time window. By following this notebook, you will learn how to set up and submit a reservation request that blocks all resources at a site, effectively securing it for maintenance or other exclusive activities.

## Step 2: Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();


## Step 3: Set the parameters

In [ ]:
site_name = "MAX"
slice_name = f"ReserveSite-{site_name}"

### Define the Time Duration
Define the duration over which resource availability will be evaluated. This duration represents the continuous period of time for which resources must be available to meet the reservation requirements. For example, in the following code snippet, a 6-hour reservation window is specified, meaning the system will search within the given `start` and `end` time range to identify a contiguous 6-hour block where resources are available for the requested slice.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

start = (datetime.now(timezone.utc) + timedelta(days=180))
end = start + timedelta(days=15)
lease_in_hours = 48

#### Step 4: Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
fablib.list_sites();

## Step 5: Create the Experiment Slice

Each site can consist of multiple hosts, each with its own resource capacity. In the following process, a node is created for each host, requesting CPU cores up to the maximum capacity that each host can provide. After configuring the slice with the necessary resources, you must submit a request to schedule and instantiate the slice within the defined time range (`start` to `end`). If resources are not available within this interval, you may need to adjust the `start` and `end` times to find an available window.

By default, the `submit` function will hold execution until the node is fully provisioned, showing real-time updates on the progress of your slice as it’s being built. This feedback allows you to monitor the slice creation process step-by-step.

In [ ]:
# Get the information for the specified site
site = fablib.resources.get_site(site_name)

# Create a new slice with the specified name
slice = fablib.new_slice(name=slice_name)

# Add a node for each host at the site, requesting all available cores on each host
idx = 0
for h in site.get_hosts().values():
    max_cores = h.get_core_capacity()
    # Add a node to the slice with the host's maximum core capacity
    node = slice.add_node(name=f"{site_name}-{idx}", host=h.get_name(), cores=max_cores/2, site=site_name)
    idx += 1  # Increment index for unique node naming
    # Add a node to the slice with the host's maximum core capacity
    node = slice.add_node(name=f"{site_name}-{idx}", host=h.get_name(), cores=max_cores/2, site=site_name)
    idx += 1  # Increment index for unique node naming

# Submit the slice request, specifying the start and end times for the lease
slice.submit(lease_start_time=start, lease_end_time=end, lease_in_hours=lease_in_hours)

## Step 5: Observe the Slice's Attributes

### Show the slice attributes 

In [ ]:
slice.show();

### List the nodes


In [ ]:
slice.list_nodes();

## Step 7: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()